In [35]:
pwd

'/home/dsc/Repositories'

In [2]:
cd ..

/home/dsc


In [3]:
cd ..

/home


In [21]:
!find -maxdepth 3 -type d -iname 'master*'

./dsc/Repositories/master-data-science


In [22]:
cd ./dsc/Repositories/master-data-science

/home/dsc/Repositories/master-data-science


In [23]:
#Este archivo tiene cambios. deberían visualizarse en la web github.

Hist_cmd/  practice/  README.md


In [1]:
!git add .


In [4]:
!git push


Counting objects: 6, done.
Delta compression using up to 4 threads.
Compressing objects: 100% (6/6), done.
Writing objects: 100% (6/6), 1.12 KiB | 286.00 KiB/s, done.
Total 6 (delta 1), reused 0 (delta 0)
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
To https://github.com/hguadalupe/master-data-science.git
   c8305bf..c5d59df  master -> master


In [36]:
cd ./master-data-science/practice/

/home/dsc/Repositories/master-data-science/practice


In [8]:
!find ../../../ -maxdepth 2 -iname 'data' 

../../../Data


In [37]:
arr_route = '../../../Data/challenge'

In [36]:
!find ../../../../Data/challenge -maxdepth 4 -type f -iname 'book*' 

../../../../Data/challenge/bookings.csv.bz2
../../../../Data/challenge/bookings.csv


In [16]:
!bzip2 -dk ../../../Data/challenge/bookings.csv.bz2


bzip2: Can't open input file ../../../Data/challenge!ls/bookings.csv.bz2: No such file or directory.


In [6]:
!ls ../../../../Data/challenge

bookings.csv  bookings.csv.bz2	searches.csv.bz2


In [10]:
pwd

'/home/dsc/Repositories/master-data-science/practice/JupyterChallenge'

### Echar un ojo:

In [9]:
!bzcat ../../../../Data/challenge/bookings.csv.bz2 | wc -l

^C


In [10]:
!head ../../../../Data/challenge/bookings.csv

head: cannot open '2' for reading: No such file or directory
==> ../../../../Data/challenge/bookings.csv <==
act_date           ^source^pos_ctry^pos_iata^pos_oid  ^rloc          ^cre_date           ^duration^distance^dep_port^dep_city^dep_ctry^arr_port^arr_city^arr_ctry^lst_port^lst_city^lst_ctry^brd_port^brd_city^brd_ctry^off_port^off_city^off_ctry^mkt_port^mkt_city^mkt_ctry^intl^route          ^carrier^bkg_class^cab_class^brd_time           ^off_time           ^pax^year^month^oid      
2013-03-05 00:00:00^1A    ^DE      ^a68dd7ae953c8acfb187a1af2dcbe123^1a11ae49fcbf545fd2afc1a24d88d2b7^ea65900e72d71f4626378e2ebd298267^2013-02-22 00:00:00^1708^0^ZRH     ^ZRH     ^CH      ^LHR     ^LON     ^GB      ^ZRH     ^ZRH     ^CH      ^LHR     ^LON     ^GB      ^ZRH     ^ZRH     ^CH      ^LHRZRH  ^LONZRH  ^CHGB    ^1^LHRZRH         ^VI^T        ^Y        ^2013-03-07 08:50:00^2013-03-07 11:33:37^-1^2013^3^NULL     
2013-03-26 00:00:00^1A    ^US      ^e612b9eeeee6f17f42d9b0d3b79e75ca^7437560d8f276

# Aquí empieza el código

In [3]:
import pandas as pd
import numpy as np
import csv
from sqlalchemy import create_engine

## Tanteos python

In [3]:
#Ejercicio 1. Recuento de líneas (versión linux)
!wc -l ../../../../Data/challenge/bookings.csv 

^C


In [1]:
file = '../../../../Data/challenge/bookings.csv'


In [18]:
#Ejercicio 1. Tantea el archivo con cuidado, para ver contenido y tamaño.
pd_opening = pd.read_csv(file, sep='^',skip_blank_lines=True,nrows=10)
!ls -sh ../../../../Data/challenge/bookings.csv
print(list(pd_opening))
print(pd_opening.head())
#MALO pd_opening = pd.read_csv('../../../../Data/challenge/bookings.csv', sep='^',skip_blank_lines=True)

type(pd_opening)

4.0G ../../../../Data/challenge/bookings.csv
['act_date           ', 'source', 'pos_ctry', 'pos_iata', 'pos_oid  ', 'rloc          ', 'cre_date           ', 'duration', 'distance', 'dep_port', 'dep_city', 'dep_ctry', 'arr_port', 'arr_city', 'arr_ctry', 'lst_port', 'lst_city', 'lst_ctry', 'brd_port', 'brd_city', 'brd_ctry', 'off_port', 'off_city', 'off_ctry', 'mkt_port', 'mkt_city', 'mkt_ctry', 'intl', 'route          ', 'carrier', 'bkg_class', 'cab_class', 'brd_time           ', 'off_time           ', 'pax', 'year', 'month', 'oid      ']
   act_date             source  pos_ctry                          pos_iata  \
0  2013-03-05 00:00:00  1A      DE        a68dd7ae953c8acfb187a1af2dcbe123   
1  2013-03-26 00:00:00  1A      US        e612b9eeeee6f17f42d9b0d3b79e75ca   
2  2013-03-26 00:00:00  1A      US        e612b9eeeee6f17f42d9b0d3b79e75ca   
3  2013-03-26 00:00:00  1A      AU        0f984b3bb6bd06661c95529bbd6193bc   
4  2013-03-26 00:00:00  1A      AU        0f984b3bb6bd06661c95529b

pandas.core.frame.DataFrame

# Observaciones
Tenemos un archivo de 4Gigas y 10 millones de filas. VAmos a tener que dividirlo en partes. 
Podemos dividirlo en "chunks" de 100000 líneas con sqlalchemy, iterando sobre el CSV.

In [6]:
%%time
df1 = pd.DataFrame()
df2 = pd.DataFrame()
df3 = pd.DataFrame()
df4 = pd.DataFrame()
for chunk in pd.read_csv(file,  sep='^',skip_blank_lines=True,chunksize=100000):
    count =0
    if count < 40:
        df1=pd.concat([df1,chunk])
        count+=1
    elif count < 80:
        df2=pd.concat([df2,chunk])
        count+=1
    elif count < 120:
        df3=pd.concat([df3,chunk])
        count+=1
    else:
        df4=pd.concat([df4,chunk])
        count+=1
        

    


KeyboardInterrupt: 

In [32]:
df1.loc[0:2,]


,act_date,source,pos_ctry,pos_iata,pos_oid,rloc,cre_date,duration,distance,dep_port,...,route,carrier,bkg_class,cab_class,brd_time,off_time,pax,year,month,oid
0,2013-03-05 00:00:00,1A,DE,a68dd7ae953c8acfb187a1af2dcbe123,1a11ae49fcbf545fd2afc1a24d88d2b7,ea65900e72d71f4626378e2ebd298267,2013-02-22 00:00:00,1708,0,ZRH,...,LHRZRH,VI,T,Y,2013-03-07 08:50:00,2013-03-07 11:33:37,-1,2013,3,NULL
1,2013-03-26 00:00:00,1A,US,e612b9eeeee6f17f42d9b0d3b79e75ca,7437560d8f276d6d05eeb806d9e7edee,737295a86982c941f1c2da9a46a14043,2013-03-26 00:00:00,135270,0,SAL,...,SALATLCLT,NV,L,Y,2013-04-12 13:04:00,2013-04-12 22:05:40,1,2013,3,NULL
2,2013-03-26 00:00:00,1A,US,e612b9eeeee6f17f42d9b0d3b79e75ca,7437560d8f276d6d05eeb806d9e7edee,737295a86982c941f1c2da9a46a14043,2013-03-26 00:00:00,135270,0,SAL,...,CLTATLSAL,NV,U,Y,2013-07-15 07:00:00,2013-07-15 11:34:51,1,2013,3,NULL


# Probando "Chunks""

In [ ]:
#Probando "Chunks""
chunksize = 10000
tp = pd.read_csv(route,chunksize=chunksize,sep='^',skip_blank_lines=True)
df = pd.concat(tp, ignore_index=True)
print (df.head())






In [9]:
pwd

'/home/dsc/Repositories/master-data-science/practice/JupyterChallenge'

In [35]:
!find ../../../../../ -maxdepth 4 './bookings.csv'

find: paths must precede expression: ./bookings.csv
Usage: find [-H] [-L] [-P] [-Olevel] [-D help|tree|search|stat|rates|opt|exec] [path...] [expression]


## Probando dicionarios

In [30]:
file = '../../../../Data/challenge/bookings.csv'
    
def csv_opener(route,chunksize,nrows):
    reader = pd.read_csv(route,chunksize=chunksize,sep='^',skip_blank_lines=True,iterator=True, nrows=nrows)
    counter=0
    nchunks=int(round(nrows/chunksize))
    dict1={}
    for chunk in reader:
        if counter <= nchunks:                            
            dict1['chunk{0}'.format(counter)]=chunk.to_dict()
            counter+=1
            return dict1
        else:
            BREAK

a=csv_opener(file,5,15)


In [33]:
print(a)

{'chunk0': {'act_date           ': {0: '2013-03-05 00:00:00', 1: '2013-03-26 00:00:00', 2: '2013-03-26 00:00:00', 3: '2013-03-26 00:00:00', 4: '2013-03-26 00:00:00'}, 'source': {0: '1A    ', 1: '1A    ', 2: '1A    ', 3: '1A    ', 4: '1A    '}, 'pos_ctry': {0: 'DE      ', 1: 'US      ', 2: 'US      ', 3: 'AU      ', 4: 'AU      '}, 'pos_iata': {0: 'a68dd7ae953c8acfb187a1af2dcbe123', 1: 'e612b9eeeee6f17f42d9b0d3b79e75ca', 2: 'e612b9eeeee6f17f42d9b0d3b79e75ca', 3: '0f984b3bb6bd06661c95529bbd6193bc', 4: '0f984b3bb6bd06661c95529bbd6193bc'}, 'pos_oid  ': {0: '1a11ae49fcbf545fd2afc1a24d88d2b7', 1: '7437560d8f276d6d05eeb806d9e7edee', 2: '7437560d8f276d6d05eeb806d9e7edee', 3: '36472c6dbaf7afec9136ac40364e2794', 4: '36472c6dbaf7afec9136ac40364e2794'}, 'rloc          ': {0: 'ea65900e72d71f4626378e2ebd298267', 1: '737295a86982c941f1c2da9a46a14043', 2: '737295a86982c941f1c2da9a46a14043', 3: '5ecf00fdcbcec761c43dc7285253d0c1', 4: '5ecf00fdcbcec761c43dc7285253d0c1'}, 'cre_date           ': {0: '2013-

In [51]:
pd.DataFrame.from_dict(a,orient='index')

,act_date,source,pos_ctry,pos_iata,pos_oid,rloc,cre_date,duration,distance,dep_port,...,route,carrier,bkg_class,cab_class,brd_time,off_time,pax,year,month,oid
chunk0,"{0: '2013-03-05 00:00:00', 1: '2013-03-26 00:0...","{0: '1A ', 1: '1A ', 2: '1A ', 3: '1A...","{0: 'DE ', 1: 'US ', 2: 'US ', ...","{0: 'a68dd7ae953c8acfb187a1af2dcbe123', 1: 'e6...","{0: '1a11ae49fcbf545fd2afc1a24d88d2b7', 1: '74...","{0: 'ea65900e72d71f4626378e2ebd298267', 1: '73...","{0: '2013-02-22 00:00:00', 1: '2013-03-26 00:0...","{0: 1708, 1: 135270, 2: 135270, 3: 30885, 4: 3...","{0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: ...","{0: 'ZRH ', 1: 'SAL ', 2: 'SAL ', ...",...,"{0: 'LHRZRH ', 1: 'SALATLCLT ', 2...","{0: 'VI', 1: 'NV', 2: 'NV', 3: 'XK', 4: 'XK', ...","{0: 'T ', 1: 'L ', 2: 'U ...","{0: 'Y ', 1: 'Y ', 2: 'Y ...","{0: '2013-03-07 08:50:00', 1: '2013-04-12 13:0...","{0: '2013-03-07 11:33:37', 1: '2013-04-12 22:0...","{0: -1, 1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1, 7:...","{0: 2013, 1: 2013, 2: 2013, 3: 2013, 4: 2013, ...","{0: 3, 1: 3, 2: 3, 3: 3, 4: 3, 5: 3, 6: 3, 7: ...","{0: 'NULL ', 1: 'NULL ', 2: 'NULL ..."


## Probando con DB

In [35]:
import pandas as pd
import numpy as np
import csv
from sqlalchemy import create_engine

In [36]:
csv_database = create_engine('sqlite:///csv_gitting.db')

In [50]:
chunksize = 5
i = 0
j = 1
for df1 in pd.read_csv(file,chunksize=chunksize,sep='^',skip_blank_lines=True,iterator=True, nrows=20):
    df1 = df1.rename(columns={c: c.replace(' ', '') for c in df1.columns})
    df1.index += j
    i+=1
    df1.to_sql('table1', csv_database, if_exists='append')
    j = df1.index[-1] + 1

In [47]:
connection = csv_database.connect()
result = connection.execute("select pos_old from coco")
for row in result:
    print("username:", row['username'])
connection.close()

OperationalError: (sqlite3.OperationalError) no such column: pos_old [SQL: 'select pos_old from coco']